In [137]:

import pandas as pd
import numpy as np
import category_encoders as ce
import json
import ast
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from statistics import mean, median
from sklearn.model_selection import train_test_split

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import sys, os
sys.path.append(os.path.join(os.path.abspath(''), '..', 'libs'))
import preparation

import importlib, sys
importlib.reload(sys.modules['preparation'])
from preparation import parsing_homeFacts, parsing_schools, parsing_year_build

In [138]:
# загрузим модель
with open('../libs/data/model_abr.pkl', 'rb') as file_model:
    model = pickle.load(file_model)


Заранее создадим функцию, которая будет расчитывать метрики:

1. MAE - средняя абсолютная ошибка (целевая метрика)
2. MAPE - средняя абсолютная ошибка в процентах 
3. $R^2$ - коэффициент детерминации 

In [139]:
def print_metrics(y_test, y_test_predict):
    print('Valid R^2: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict)))
    print('Valid MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))
    print('Valid MAPE: {:.3f}'.format(metrics.mean_absolute_percentage_error(y_test, y_test_predict)*100))

Подгрузим константы и бинарный справочник

In [140]:
with open('../libs/data/const_dict.pkl', 'rb') as pkl_file:
    const_dict = pickle.load(pkl_file)
    
with open('../libs/data/dict_binary.pkl', 'rb') as pkl_file:
    dict_binary = pickle.load(pkl_file)

In [141]:
X_valid = pd.read_csv('data/df_X_valid.csv', dtype={"zipcode":str})
y_valid = pd.read_csv('data/df_y_valid.csv')

Признаки 'status', 'PrivatePool'

In [142]:
X_valid = X_valid.drop(['status'], axis=1)

X_valid['PrivatePool'] = X_valid['PrivatePool'].fillna(X_valid['private pool'])
X_valid['PrivatePool'] = X_valid['PrivatePool'].apply(lambda x: 1 if x=='Yes' or x=='yes' else 0)

X_valid = X_valid.drop(['private pool'], axis=1)

Признак 'propertyType'. Пустые значения заполняем из справочника констант, по имени признака

In [143]:
X_valid['propertyType'] = X_valid['propertyType'].fillna(const_dict['propertyType'])
X_valid['propertyType'] = X_valid['propertyType'].str.lower()
X_valid['propertyType'] = X_valid['propertyType'].apply(lambda x: x+' other')

lst_propertyType = ['single', 'condo', ['land', 'lot'], ['townhouse','townhome'], 'multi', 'traditional', 
                    ['coop', 'co-op'], ['ranch', 'farm'], 'high rise', ['low-rise', 'low'], 'detached', 'mobile', ['contemporary','modern'],
                    ['1 story', '1', 'one story'], ['2 story', '2', '2 stories', 'two stories', 'two story', 'stories'], ['3 stor', '3'], ['colonial', 'transitional', 'historical'],
                    ['garden', 'cluster home'], 
                    ['craft', 'cottage', 'tri-level', 'bungalow', 'cape', 'spanish', 'mediterranean', 'victorian', 'florida', 'french', 'georgian', 'loft', 'art', 'tudor'],
                    ['other', 'custom', 'manufactured']]

mask_nan = X_valid['propertyType'].isna()

for item in lst_propertyType:
    if type(item)==list:
        for ind, prprt in enumerate(item):
            X_valid['propertyType'].where(~(X_valid[~mask_nan].propertyType.str.contains(prprt) ), other=item[0], inplace=True)
    else:

        X_valid['propertyType'].where(~(X_valid[~mask_nan].propertyType.str.contains(item) ), other=item, inplace=True) 

Признаки 'baths', 'beds', 'fireplace' 

In [144]:
# Преобразуем колонку 'baths' 
pattern = r'(\d*\.\d+|\d+)'
X_valid["baths"] = pd.to_numeric(X_valid["baths"].str.extract(pattern)[0], downcast='float')
X_valid["baths"] = X_valid["baths"].fillna(const_dict['baths'])         
X_valid["baths"] = X_valid["baths"].fillna(0)

X_valid["beds"] = pd.to_numeric(X_valid["beds"].str.extract(pattern)[0], downcast='float')
X_valid["beds"] = X_valid["beds"].fillna(const_dict['beds'])    
X_valid["beds"] = X_valid["beds"].fillna(0)

X_valid['fireplace'] = X_valid['fireplace'].fillna('0')
X_valid['fireplace'] = X_valid['fireplace'].apply(lambda x: 0 if x.lower() in ['0', 'no', 'not applicable'] else 1)

# Удалим лишнии колонки
X_valid = X_valid.drop(columns=['street', 'stories', 'mls-id', 'MlsId'], axis=1)

Распарсим признаки 'homeFacts', 'school'

In [145]:
X_valid = preparation.parsing_homeFacts(X_valid)
X_valid = preparation.parsing_schools(X_valid)

Признаки 'Year built', 'Remodeled year'

In [146]:
#Year built    
X_valid['Year built'] = X_valid['Year built'].replace('',np.nan)
X_valid['Year built'] = X_valid['Year built'].replace('No Data',np.nan)
pattern = r'(^\d{4}$)'
X_valid['Year built'] = X_valid['Year built'].str.extract(pattern)[0]
X_valid['Year built'].fillna(const_dict['Year built'], inplace=True)    
# парсим значения в колонке 'Year build' и записываем новые значения в колонку 'interval_year'
X_valid = preparation.parsing_year_build(X_valid)
# Remodeled year
X_valid['Remodeled year'] = X_valid['Remodeled year'].str.extract(pattern)[0]
X_valid['Remodeled year'] = X_valid['Remodeled year'].fillna(0)
X_valid['Remodeled year'] = X_valid['Remodeled year'].apply(lambda x: 0 if x==0 else 1)

Признаки 'Price/sqft' и 'sqft'

In [147]:
X_valid['price/sqft'] = X_valid['Price/sqft'].str.replace(',','')
pattern = r'\b(\d+)\b'
X_valid['price/sqft'] = pd.to_numeric(X_valid['price/sqft'].str.extract(pattern)[0])

X_valid['sqft'] = X_valid['sqft'].str.replace(',','')
X_valid['sqft'] = pd.to_numeric(X_valid['sqft'].str.extract(pattern)[0])

X_valid["price/sqft"] = X_valid["price/sqft"].fillna(const_dict['price/sqft'])    # df["price/sqft"].median()
X_valid["sqft"] = X_valid["sqft"].fillna(const_dict['sqft'])   # df["sqft"].median()

In [148]:
# Удалим лишнии колонки
X_valid = X_valid.drop(columns=['lotsize', 'Price/sqft'], axis=1)

Признаки 'Cooling', 'Heating' и 'Parking'

In [149]:
X_valid['Cooling'] = X_valid['Cooling'].fillna('0') 
X_valid['Cooling'] = X_valid['Cooling'].apply(lambda x: 0 if x.lower() in ['','no data', 'none'] else 1)

X_valid['Heating'] = X_valid['Heating'].fillna('0') 
X_valid['Heating'] = X_valid['Heating'].apply(lambda x: 0 if x.lower() in ['','no data', 'none'] else 1)

X_valid['Parking'] = X_valid['Parking'].fillna('0')
X_valid['Parking'] = X_valid['Parking'].apply(lambda x: 0 if x.lower() in ['','no data', 'none'] else 1)

Кодируем бинарные признаки


In [150]:
def get_binary_value(value, ind, dict_bin, value_mode):
    
    if value in dict_bin.keys():
        return dict_bin[value][ind]
    else:
        return dict_bin[value_mode][ind]
    
   
dict_zipcode = dict_binary['zipcode']
for ind, col_name in enumerate(dict_zipcode['col_names']):
    X_valid[col_name] = X_valid['zipcode'].apply(lambda x: get_binary_value(x, ind, dict_zipcode, const_dict['zipcode'+'_mode']))
    
dict_city = dict_binary['city']
for ind, col_name in enumerate(dict_city['col_names']):
    X_valid[col_name] = X_valid['city'].apply(lambda x: get_binary_value(x, ind, dict_city, const_dict['city'+'_mode']))
    

dict_state = dict_binary['state']
for ind, col_name in enumerate(dict_state['col_names']):
    X_valid[col_name] = X_valid['state'].apply(lambda x: get_binary_value(x, ind, dict_state, const_dict['state'+'_mode']))
    
dict_propertyType = dict_binary['propertyType']
for ind, col_name in enumerate(dict_propertyType['col_names']):
    X_valid[col_name] = X_valid['propertyType'].apply(lambda x: get_binary_value(x, ind, dict_propertyType, const_dict['propertyType'+'_mode']))
    
dict_interval_year = dict_binary['interval_year']
for ind, col_name in enumerate(dict_interval_year['col_names']):
    X_valid[col_name] = X_valid['interval_year'].apply(lambda x: get_binary_value(x, ind, dict_interval_year, const_dict['interval_year'+'_mode']))   


In [151]:
# Удалим лишнии колонки
X_valid = X_valid.drop(columns=['zipcode', 'city', 'state', 'propertyType', 'interval_year'], axis=1)

Нормализуем признаки

In [152]:


col_scale = ['baths', 'sqft', 'beds', 'Elementary', 'Middle', 'High', 'Other', 'Rating', 'Distance', 'price/sqft']
scaler = const_dict['MinMaxScaler_model']
 
X_valid[col_scale] = scaler.transform(X_valid[col_scale].values)

In [153]:
y_valid_pred = model.predict(X_valid[const_dict['order_columns_name']])

#Выводим результирующие метрики
print_metrics(y_valid, y_valid_pred)

Valid R^2: 0.762
Valid MAE: 86218.849
Valid MAPE: 22.291


### Все манипуляции с валидационном файлом вынесены в отдельную функцию 'preparate_file' модуля 'preparation' ###

Поэтому всё, что было выполнено выше, можно записать в несколько строчек

In [154]:
X_valid = pd.read_csv('data/df_X_valid.csv', dtype={"zipcode":str})
y_valid = pd.read_csv('data/df_y_valid.csv')

X_valid = preparation.preparate_file(X_valid, method_clean=False)

y_valid_pred = model.predict(X_valid[const_dict['order_columns_name']])

#Выводим результирующие метрики
print_metrics(y_valid, y_valid_pred)

Valid R^2: 0.762
Valid MAE: 86218.849
Valid MAPE: 22.291


Метрика MAPE на валидационном файле показала значение 22.291%, что немного больше значения метрики MAPE на тестовых данных (11.858%). Это объясняется следующим:
1. Разный размер тестового и валидационного наборов (67828 и  1769 записей соответственно)  
2. Данные в тестовой выборке были очищены на предмет выбрасов в таких признаках как 'sqft', 'price/sqft' (интервальная очистка и z-отклонения). Если провести такую очистку данных в валидационной выборки (удаляя строки в выборке), то значение MAPE валидационного набора уменьшится, но при этом не будет предсказания по 41 записи из 1769 записей.  

In [156]:
X_valid = pd.read_csv('data/df_X_valid.csv', dtype={"zipcode":str})
y_valid = pd.read_csv('data/df_y_valid.csv')
df = pd.concat([X_valid, y_valid], axis=1)
print('Число записей в валидационной выборке', df.shape[0])
df_val = preparation.preparate_file(df, method_clean=True)
print('Число записей в валидационной выборке после очистки', df_val.shape[0])

X_valid = df_val[const_dict['order_columns_name']]
y_valid = df_val['target']
y_valid_pred = model.predict(X_valid)
#Выводим результирующие метрики
print_metrics(y_valid, y_valid_pred)

Число записей в валидационной выборке 1769
Число записей в валидационной выборке после очистки 1728
Valid R^2: 0.766
Valid MAE: 73509.997
Valid MAPE: 17.701


После очистки данных валидационного набора значение метрики MAPE уменьшилось с **22.291%** до **17.701%**

### **Примечание** ###
Предположу, что можно уменьшить процент ошибки, если прибегнуть к внешним данным по GPS координатам и распарсить признак 'street'. Но в свободном (бесплатном) доступе я не нашел таких данных